# Homework #3: PCA/Hyperparameter/CV
Data source: http://archive.ics.uci.edu/ml/datasets/Polish+companies+bankruptcy+data

In [1]:
from scipy.io import arff
import pandas as pd
import numpy as np

In [2]:
data = arff.loadarff('4year.arff')
df = pd.DataFrame(data[0])

In [3]:
df['bankruptcy'] = (df['class']==b'1')
df.drop(columns=['class'], inplace=True)
df.columns = ['X{0:02d}'.format(k) for k in range(1,65)] + ['bankruptcy']

In [4]:
for i in range(len(df['bankruptcy'])):
    df.loc[i, 'bankruptcy'] = 0 if df.loc[i,'bankruptcy'] == False else 1

In [5]:
df.describe()

X01          X02          X03           X04           X05  \
count  9791.000000  9791.000000  9791.000000   9749.000000  9.771000e+03   
mean      0.043019     0.596404     0.130959      8.136600  6.465164e+01   
std       0.359321     4.587122     4.559074    290.647281  1.475939e+04   
min     -12.458000     0.000000  -445.910000     -0.045319 -3.794600e+05   
25%       0.001321     0.263145     0.020377      1.047000 -5.121700e+01   
50%       0.041364     0.467740     0.199290      1.591800 -5.557600e-02   
75%       0.111130     0.689255     0.410670      2.880400  5.573200e+01   
max      20.482000   446.910000    22.769000  27146.000000  1.034100e+06   

               X06          X07           X08          X09          X10  ...  \
count  9791.000000  9791.000000   9773.000000  9792.000000  9791.000000  ...   
mean     -0.059273     0.059446     19.884016     1.882296     0.389040  ...   
std       6.812754     0.533344    698.697015    17.674650     4.590299  ...   
min    -486.820000   -12.458000     -1.848200    -0.032371  -445.910000  ...   
25%      -0.000578     0.003004      0.428300     1.006675     0.294440  ...   
50%       0.000000     0.048820      1.088700     1.161300     0.510450  ...   
75%       0.065322     0.126940      2.691000     1.970225     0.714290  ...   
max     322.200000    38.618000  53209.000000  1704.800000    12.602000  ...   

                X55          X56          X57          X58          X59  \
count  9.792000e+03  9771.000000  9791.000000  9776.000000  9791.000000   
mean   7.686330e+03    -0.992263     0.035022     1.133287     0.856053   
std    7.605261e+04    77.007971     8.945365     8.038201    26.393305   
min   -7.132200e+05 -7522.100000  -597.420000   -30.892000  -284.380000   
25%    2.184000e+01     0.003121     0.008768     0.885722     0.000000   
50%    9.503300e+02     0.043679     0.098026     0.958305     0.002129   
75%    4.694550e+03     0.117170     0.242680     0.996163     0.211790   
max    6.123700e+06   112.020000   226.760000   668.750000  1661.000000   

                 X60            X61           X62          X63           X64  
count    9178.000000    9760.000000  9.771000e+03  9749.000000   9561.000000  
mean      118.156064      25.194430  2.015157e+03     8.660813     35.949619  
std      3230.316692    1099.260821  1.171461e+05    60.838202    483.318623  
min         0.000000     -12.656000 -1.496500e+04    -0.024390     -0.000015  
25%         5.356325       4.267700  4.323400e+01     2.938800      2.012900  
50%         9.482000       6.283550  7.472900e+01     4.848900      4.041600  
75%        19.506000       9.938200  1.233450e+02     8.363800      9.413500  
max    251570.000000  108000.000000  1.077900e+07  5662.400000  21153.000000  

[8 rows x 64 columns]

In [6]:
sum(df.bankruptcy == True)

515

In [7]:
df.fillna(df.mean(), inplace=True)
df.isna().sum()
X_imp = df.values

In [8]:
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

X, y = X_imp[:, :-1], X_imp[:, -1]

X_train, X_test, y_train, y_test =\
    train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

In [9]:
import sklearn.preprocessing as skpre

stdsc = skpre.StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
print(X_train_std.shape)
X_test_std = stdsc.transform(X_test)
print(X_test_std.shape)

(6854, 64)
(2938, 64)


## Extract 3 features using PCA method

In [10]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [11]:
pca = PCA(n_components=3)
X_train_pca = pca.fit_transform(X_train_std)
X_test_pca = pca.transform(X_test_std)

### LR

In [12]:
lr = LogisticRegression(C=0.01, random_state=0, penalty='l1')
lr.fit(X_train_pca, y_train)
print('Training accuracy: %.4f' % lr.score(X_train_pca, y_train))
print('Test accuracy: %.4f' % lr.score(X_test_pca, y_test))

Training accuracy: 0.9473
Test accuracy: 0.9462


### SVM

In [13]:
svm = SVC(C=0.01, random_state=0, kernel='rbf')
svm.fit(X_train_pca,y_train)
print('Training accuracy: %.4f' % svm.score(X_train_pca, y_train))
print('Test accuracy: %.4f'% svm.score(X_test_pca, y_test))

Training accuracy: 0.9475
Test accuracy: 0.9472


### Decision Tree

In [14]:
tree = DecisionTreeClassifier(criterion='gini', max_depth=2)
tree.fit(X_train_pca,y_train)
print('Training accuracy: %.4f' % tree.score(X_train_pca, y_train))
print('Test accuracy: %.4f' % tree.score(X_test_pca, y_test))

Training accuracy: 0.9475
Test accuracy: 0.9472


## PCA using pipeline


### LR

In [15]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

pipe_lr = make_pipeline(StandardScaler(), PCA(n_components=3), 
                        LogisticRegression(C=0.01, random_state=0, penalty='l1'))
pipe_lr.fit(X_train, y_train)
print('Training accuracy: %.4f' % pipe_lr.score(X_train, y_train))
print('Test accuracy: %.4f' % pipe_lr.score(X_test, y_test))

Training accuracy: 0.9473
Test accuracy: 0.9462


### SVM

In [16]:
pipe_svm = make_pipeline(StandardScaler(), PCA(n_components=3),
                         SVC(C=0.01, random_state=0, kernel='rbf'))
pipe_svm.fit(X_train, y_train)
print('Training accuracy: %.4f' % pipe_svm.score(X_train, y_train))
print('Test accuracy: %.4f' % pipe_svm.score(X_test, y_test))

Training accuracy: 0.9475
Test accuracy: 0.9472


### Decision Tree

In [17]:
pipe_tree = make_pipeline(StandardScaler(), PCA(n_components=3),
                          DecisionTreeClassifier(criterion='gini', max_depth=2))
pipe_tree.fit(X_train, y_train)
print('Training accuracy: %.4f' % pipe_tree.score(X_train, y_train))
print('Test accuracy: %.4f' % pipe_tree.score(X_test, y_test))

Training accuracy: 0.9475
Test accuracy: 0.9472


## 5 fold Cross-Validation

In [18]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

### LR

In [19]:
kfold = StratifiedKFold(n_splits=5).split(X_train, y_train)
pipe_lr_cv = make_pipeline(StandardScaler(), PCA(n_components=3), 
                        LogisticRegression(C=0.01, random_state=0, penalty='l1'))
score_lr_cv = []
for k, (train, test) in enumerate(kfold):
    pipe_lr_cv.fit(X_train[train], y_train[train])
    score = pipe_lr_cv.score(X_train[test], y_train[test])
    score_lr_cv.append(score)

print('CV accuracy of LR:', score_lr_cv)

CV accuracy of LR: [0.9474835886214442, 0.9474835886214442, 0.9474835886214442, 0.9474835886214442, 0.9467153284671533]


In [20]:
cv_score_lr = cross_val_score(estimator=pipe_lr, X=X_train, y=y_train, cv=5)
print('CV accuracy of LR:', cv_score_lr)

CV accuracy of LR: [0.94748359 0.94748359 0.94748359 0.94748359 0.94671533]


### SVM

In [21]:
kfold = StratifiedKFold(n_splits=5).split(X_train, y_train)
pipe_svm_cv = make_pipeline(StandardScaler(), PCA(n_components=3),
                         SVC(C=0.01, random_state=0, kernel='rbf'))
score_svm_cv = []
for k, (train, test) in enumerate(kfold):
    pipe_svm_cv.fit(X_train[train], y_train[train])
    score = pipe_svm_cv.score(X_train[test], y_train[test])
    score_svm_cv.append(score)
print('CV accuracy of SVM:', score_svm_cv)

CV accuracy of SVM: [0.9474835886214442, 0.9474835886214442, 0.9474835886214442, 0.9474835886214442, 0.9474452554744526]


In [22]:
cv_score_svm = cross_val_score(estimator=pipe_svm, X=X_train, y=y_train, cv=5)
print('CV accuracy of SVM:', cv_score_svm)

CV accuracy of SVM: [0.94748359 0.94748359 0.94748359 0.94748359 0.94744526]


### Decision Tree

In [23]:
kfold = StratifiedKFold(n_splits=5).split(X_train, y_train)
pipe_tree_cv = make_pipeline(StandardScaler(), PCA(n_components=3),
                          DecisionTreeClassifier(criterion='gini', max_depth=2))
score_tree_cv = []
for k, (train, test) in enumerate(kfold):
    pipe_tree_cv.fit(X_train[train], y_train[train])
    score = pipe_tree_cv.score(X_train[test], y_train[test])
    score_tree_cv.append(score)
print('CV accuracy of Decision Tree:', score_tree_cv)

CV accuracy of Decision Tree: [0.9474835886214442, 0.9474835886214442, 0.9474835886214442, 0.9474835886214442, 0.9474452554744526]


In [24]:
cv_score_tree = cross_val_score(estimator=pipe_tree, X=X_train, y=y_train, cv=5)
print('CV accuracy of Decision Tree:', cv_score_tree)

CV accuracy of Decision Tree: [0.94748359 0.94748359 0.94748359 0.94748359 0.94744526]


## Grid search

### LR

In [25]:
from sklearn.model_selection import GridSearchCV

para_range = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
para_grid = [{'logisticregression__C': para_range, 
              'logisticregression__penalty': ['l1','l2']}]
gs_lr = GridSearchCV(estimator=pipe_lr, param_grid=para_grid, scoring='accuracy', cv=5)
gs_lr = gs_lr.fit(X_train, y_train)

In [26]:
print('%.4f' % gs_lr.best_score_)

0.9475


In [27]:
print(gs_lr.best_params_)

{'logisticregression__C': 0.0001, 'logisticregression__penalty': 'l1'}


### SVM

In [ ]:
para_range = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
# para_grid = [{'svc__C': para_range, 'svc__kernel': ['linear', 'rbf']}]
para_grid = [{'svc__C': para_range, 'svc__kernel': ['linear']}]
gs_svm = GridSearchCV(estimator=pipe_svm, param_grid=para_grid, scoring='accuracy', cv=5)
gs_svm = gs_svm.fit(X_train, y_train)

In [ ]:
print('%.4f' % gs_svm.best_score_)

In [ ]:
print(gs_svm.best_params_)

### Decision Tree

In [28]:
para_depth = np.arange(1, 10)
para_grid = [{'decisiontreeclassifier__max_depth': para_depth,
               'decisiontreeclassifier__criterion': ['gini', 'entropy']}]
gs_tree = GridSearchCV(estimator=pipe_tree, param_grid=para_grid, scoring='accuracy', cv=5)
gs_tree = gs_tree.fit(X_train, y_train)

In [29]:
print('%.4f' % gs_tree.best_score_)

0.9475


In [30]:
print(gs_tree.best_params_)

{'decisiontreeclassifier__criterion': 'gini', 'decisiontreeclassifier__max_depth': 1}
